In [1]:
# selenium 4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup 
import pandas as pd
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.maximize_window()
driver.implicitly_wait(10)
driver.set_page_load_timeout(5)

In [2]:
df = pd.read_csv('match_results.csv')
link = df.sample(1)['score_card_link'].iloc[0]

In [3]:
driver.get(link)

In [4]:
try:
    junk_click_box = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH,'//*[@id="wzrk-cancel"]'))
    )
    junk_click_box.click()
except NoSuchElementException:
    print('No Such element')

In [5]:
driver.delete_all_cookies()